####**dependencies + packages**

In [1]:
!apt install gdal-bin python-gdal python3-gdal 
# install rtree - geopandas requirement
!apt install python3-rtree 
# install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# install descartes - geopandas requirement
!pip install descartes 
# install vincenty
!pip install vincenty

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
python3-gdal is already the newest version (2.2.3+dfsg-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-resources
Suggested packages:
  python3-setuptools
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-

In [2]:
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from vincenty import vincenty

import csv
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys

plt.style.use('ggplot')

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


####**functions**

In [5]:
def calc_distance(df):
  distances = []

  for idx, row in df.iterrows():
    distances.append(vincenty((row[1].coords[0][0], row[1].coords[0][1]), (row[2].coords[0][0], row[2].coords[0][1])))

  distance_series = pd.Series(distances)

  return distance_series

# create bins
def create_bins(series, bins, labels):
  return pd.cut(series, bins = bins, labels = labels)

def merge_series_df(df, series_list, col_rename):
  for i in range(len(series_list)):
    df_series = pd.Series(series_list[i])
    df = pd.concat([df, df_series], axis = 1, sort= False)
    df = df.rename(columns= {0 : col_rename[i]})

  return df

####**load datasets**

In [6]:
# change directory
drive.mount('/content/drive', force_remount= True)
os.chdir('/content/drive/My Drive/MRP/to_be_submitted')

Mounted at /content/drive


In [7]:
mtl_pts_json = 'mtl_pts_2017_MODEL_MRP.geojson'

# import file
mtl_pts = gpd.read_file(mtl_pts_json)
mtl_pts.head()

,id_trip,id_trip_seq,timestamp,DAUID,CTUID,weather_station_id,distance,speed,acceleration,jerk,bearing,geometry
0,150744,0,2017-09-18T10:03:52,24660491,4620266.00,10761,0.062391,8.31,0.000354,-0.000130,0.001470,MULTIPOINT (-73.64608 45.54617)
1,150744,1,2017-09-18T10:03:57,24660491,4620266.00,10761,0.085496,13.60,-0.000294,-0.000050,0.000854,MULTIPOINT (-73.64534 45.54595)
2,150744,2,2017-09-18T10:04:03,24660491,4620266.00,10761,0.062440,14.79,-0.000594,0.000227,0.000819,MULTIPOINT (-73.64433 45.54566)
3,150744,3,2017-09-18T10:04:08,24660491,4620266.00,10761,0.066643,11.87,0.000542,-0.000072,0.000596,MULTIPOINT (-73.64359 45.54545)
4,150744,4,2017-09-18T10:04:15,24660491,4620266.00,10761,0.053257,11.63,0.000040,-0.000164,0.000135,MULTIPOINT (-73.64280 45.54523)


In [8]:
print ('no. of unique trips : {}'.format(len(mtl_pts['id_trip'].unique())))
print ('no. of id_trip_seq [0] : {}'.format(len(mtl_pts[mtl_pts['id_trip_seq'] == 0])))

no. of unique trips : 46568
no. of id_trip_seq [0] : 46568


In [9]:
# import transport mode to join with mtl_pts
mtl_trajet_transport_mode = 'mtl_trajet_2017_transport_mode_MRP.csv'

# import file
mtl_trajet_mode = pd.read_csv(mtl_trajet_transport_mode, index_col = 0)
mtl_trajet_mode.head()

,id_trip,mode,purpose
3,150744,automobile,Reconduire / aller chercher une personne
13,173651,automobile,Travail / Rendez-vous d'affaires
16,149086,public_transportation,Travail / Rendez-vous d'affaires
19,358412,walk,Travail / Rendez-vous d'affaires
20,350121,automobile,Travail / Rendez-vous d'affaires


In [10]:
# translate from french to english
print ('purpose of trip (unique) : \n{}'.format(mtl_trajet_mode['purpose'].unique()))
print ('\nvalue counts (fr):\n {}'.format(mtl_trajet_mode['purpose'].value_counts()))

purpose_fr = ['Reconduire / aller chercher une personne', "Travail / Rendez-vous d'affaires", 'Magasinage / emplettes', 
              'Retourner à mon domicile', 'Santé', 'Loisir', 'Éducation', 'Autre', 'Repas / collation / café', 'ND']
purpose_en = ['drive / pick up a person', 'work / business meeting', 'shopping / shopping', 
              'return to my home', 'health', 'leisure', 'education', 'other', 'meal / snack / coffee', 'NA']

mtl_trajet_mode['purpose'].replace(to_replace = purpose_fr, value = purpose_en, inplace = True)

# ensure that value counts are the same
print ('\nvalue counts (en):\n {}'.format(mtl_trajet_mode['purpose'].value_counts()))

purpose of trip (unique) : 
['Reconduire / aller chercher une personne'
 "Travail / Rendez-vous d'affaires" 'Magasinage / emplettes'
 'Retourner à mon domicile' 'Santé' 'Loisir' 'Éducation' 'Autre'
 'Repas / collation / café' 'ND']

value counts (fr):
 Retourner à mon domicile                    22882
Travail / Rendez-vous d'affaires            16079
Loisir                                       8418
Magasinage / emplettes                       8001
Repas / collation / café                     2875
Éducation                                    2205
Reconduire / aller chercher une personne     1462
Autre                                        1068
Santé                                         952
ND                                             20
Name: purpose, dtype: int64

value counts (en):
 return to my home           22882
work / business meeting     16079
leisure                      8418
shopping / shopping          8001
meal / snack / coffee        2875
education                   

In [11]:
# merge transport mode
mtl_pts = mtl_pts.merge(mtl_trajet_mode, how = 'left', on = 'id_trip')
mtl_pts.head()

,id_trip,id_trip_seq,timestamp,DAUID,CTUID,weather_station_id,distance,speed,acceleration,jerk,bearing,geometry,mode,purpose
0,150744,0,2017-09-18T10:03:52,24660491,4620266.00,10761,0.062391,8.31,0.000354,-0.000130,0.001470,MULTIPOINT (-73.64608 45.54617),automobile,drive / pick up a person
1,150744,1,2017-09-18T10:03:57,24660491,4620266.00,10761,0.085496,13.60,-0.000294,-0.000050,0.000854,MULTIPOINT (-73.64534 45.54595),automobile,drive / pick up a person
2,150744,2,2017-09-18T10:04:03,24660491,4620266.00,10761,0.062440,14.79,-0.000594,0.000227,0.000819,MULTIPOINT (-73.64433 45.54566),automobile,drive / pick up a person
3,150744,3,2017-09-18T10:04:08,24660491,4620266.00,10761,0.066643,11.87,0.000542,-0.000072,0.000596,MULTIPOINT (-73.64359 45.54545),automobile,drive / pick up a person
4,150744,4,2017-09-18T10:04:15,24660491,4620266.00,10761,0.053257,11.63,0.000040,-0.000164,0.000135,MULTIPOINT (-73.64280 45.54523),automobile,drive / pick up a person


In [12]:
# transport mode counts
mtl_pts['mode'].value_counts()

automobile               1270118
bike                      698972
public_transportation     375664
walk                      192682
other                       7841
na                          5417
Name: mode, dtype: int64

In [13]:
# remove id_trip with other/na
remove_mode = ['na', 'other']

remove_ids = mtl_pts['id_trip'][mtl_pts['mode'].isin(remove_mode)].unique()
print ('no. of id_trips to remove : {}'.format(len(remove_ids))) 

no. of id_trips to remove : 257


In [14]:
# remove records
mtl_pts = mtl_pts[~mtl_pts['id_trip'].isin(remove_ids)]

# check values counts to verify id_trips have been removed
mtl_pts['mode'].value_counts()

automobile               1270118
bike                      698972
public_transportation     375664
walk                      192682
Name: mode, dtype: int64

In [15]:
# query id_trip/mode
mtl_pts_id_trip_mode = mtl_pts[['id_trip', 'mode', 'purpose']].drop_duplicates().reset_index(drop = True)
mtl_pts_id_trip_mode.head()

,id_trip,mode,purpose
0,150744,automobile,drive / pick up a person
1,173651,automobile,work / business meeting
2,149086,public_transportation,work / business meeting
3,358412,walk,work / business meeting
4,35763,automobile,shopping / shopping


In [16]:
# ensure that there are no duplicate id_trips
mtl_pts_id_trip_mode['id_trip'].value_counts(ascending = False)

67583     1
418464    1
5448      1
359749    1
30020     1
         ..
424513    1
244418    1
240320    1
150072    1
2049      1
Name: id_trip, Length: 46311, dtype: int64

In [17]:
mtl_pts_id_trip_mode['mode'].value_counts()

automobile               20982
bike                     11520
public_transportation     8431
walk                      5378
Name: mode, dtype: int64

In [18]:
mtl_pts_id_trip_mode['purpose'].value_counts()

return to my home           17264
work / business meeting     12303
leisure                      6060
shopping / shopping          5306
meal / snack / coffee        1596
education                    1399
drive / pick up a person     1030
health                        688
other                         657
NA                              8
Name: purpose, dtype: int64

In [19]:
# explore distribution of mode/purpose
pd.crosstab(mtl_pts_id_trip_mode['mode'], mtl_pts_id_trip_mode['purpose'])

purpose,NA,drive / pick up a person,education,health,leisure,meal / snack / coffee,other,return to my home,shopping / shopping,work / business meeting
mode,,,,,,,,,,
automobile,3,683,258,330,2979,717,319,7621,3333,4739
bike,2,124,454,132,1194,254,129,4603,640,3988
public_transportation,3,95,544,106,729,172,134,3407,401,2840
walk,0,128,143,120,1158,453,75,1633,932,736


In [20]:
# integer encode labels in order to run one hot encoding prior to modeling
# covert to array
encode_mode = np.array(mtl_pts_id_trip_mode['mode'])
encode_purpose = np.array(mtl_pts_id_trip_mode['purpose'])

# encode string values into integer
label_encoder = LabelEncoder()
int_encoded_mode = label_encoder.fit_transform(encode_mode)
int_encoded_purpose = label_encoder.fit_transform(encode_purpose)

int_encoded_purpose

array([1, 9, 9, ..., 7, 7, 7])

In [21]:
# merge with mtl_pts_id_trip_mode
mtl_pts_id_trip_mode['mode_label'] = int_encoded_mode
mtl_pts_id_trip_mode['purpose_label'] = int_encoded_purpose
mtl_pts_id_trip_mode.head()

,id_trip,mode,purpose,mode_label,purpose_label
0,150744,automobile,drive / pick up a person,0,1
1,173651,automobile,work / business meeting,0,9
2,149086,public_transportation,work / business meeting,2,9
3,358412,walk,work / business meeting,3,9
4,35763,automobile,shopping / shopping,0,8


In [22]:
# verify unique values
mtl_pts_id_trip_mode[['mode', 'mode_label']].apply(pd.Series.unique)

,mode,mode_label
0,automobile,0
1,public_transportation,2
2,walk,3
3,bike,1


In [23]:
# verify unique values
mtl_pts_id_trip_mode[['purpose', 'purpose_label']].apply(pd.Series.unique)

,purpose,purpose_label
0,drive / pick up a person,1
1,work / business meeting,9
2,shopping / shopping,8
3,return to my home,7
4,health,3
5,leisure,4
6,education,2
7,other,6
8,meal / snack / coffee,5
9,NA,0


In [24]:
mtl_pts_id_trip_mode.head()

,id_trip,mode,purpose,mode_label,purpose_label
0,150744,automobile,drive / pick up a person,0,1
1,173651,automobile,work / business meeting,0,9
2,149086,public_transportation,work / business meeting,2,9
3,358412,walk,work / business meeting,3,9
4,35763,automobile,shopping / shopping,0,8


In [25]:
# get trip start time / day
mtl_pts_trip_start = mtl_pts[['id_trip', 'id_trip_seq', 'timestamp']][mtl_pts['id_trip_seq'] == 0].reset_index(drop = True)
mtl_pts_trip_start.head()

,id_trip,id_trip_seq,timestamp
0,150744,0,2017-09-18T10:03:52
1,173651,0,2017-09-18T11:14:25
2,149086,0,2017-09-18T11:22:11
3,358412,0,2017-09-18T11:29:54
4,35763,0,2017-09-18T11:33:11


In [26]:
# get date attributes
# convert to timestamp
mtl_pts_trip_start['timestamp'] = pd.to_datetime(mtl_pts_trip_start['timestamp'])
mtl_pts_trip_start['dayofweek'] = mtl_pts_trip_start['timestamp'].dt.dayofweek
mtl_pts_trip_start['weekend'] = np.where(mtl_pts_trip_start['dayofweek'] >= 5, 1, 0)
mtl_pts_trip_start['start_hour'] = mtl_pts_trip_start['timestamp'].dt.hour
mtl_pts_trip_start['start_min'] = mtl_pts_trip_start['timestamp'].dt.minute

In [27]:
# create bins for start_min_interval
mtl_pts_trip_start['start_min_interval'] = create_bins(mtl_pts_trip_start['start_min'], [-1, 14, 29, 44, 60], [0, 15, 30, 45])

# create bins for start_min_interval
mtl_pts_trip_start['start_hour_interval'] = create_bins(mtl_pts_trip_start['start_hour'], 
                                                        [-1, 2, 5, 8, 11, 14, 17, 20, 23], 
                                                        [0, 3, 6, 9, 12, 15, 18, 21])


mtl_pts_trip_start.head()

,id_trip,id_trip_seq,timestamp,dayofweek,weekend,start_hour,start_min,start_min_interval,start_hour_interval
0,150744,0,2017-09-18 10:03:52,0,0,10,3,0,9
1,173651,0,2017-09-18 11:14:25,0,0,11,14,0,9
2,149086,0,2017-09-18 11:22:11,0,0,11,22,15,9
3,358412,0,2017-09-18 11:29:54,0,0,11,29,15,9
4,35763,0,2017-09-18 11:33:11,0,0,11,33,30,9


In [28]:
# merge start data with trip data
mtl_pts_id_trip_mode = pd.merge(mtl_pts_id_trip_mode, mtl_pts_trip_start, how = 'left', on = 'id_trip')
mtl_pts_id_trip_mode.head()

,id_trip,mode,purpose,mode_label,purpose_label,id_trip_seq,timestamp,dayofweek,weekend,start_hour,start_min,start_min_interval,start_hour_interval
0,150744,automobile,drive / pick up a person,0,1,0,2017-09-18 10:03:52,0,0,10,3,0,9
1,173651,automobile,work / business meeting,0,9,0,2017-09-18 11:14:25,0,0,11,14,0,9
2,149086,public_transportation,work / business meeting,2,9,0,2017-09-18 11:22:11,0,0,11,22,15,9
3,358412,walk,work / business meeting,3,9,0,2017-09-18 11:29:54,0,0,11,29,15,9
4,35763,automobile,shopping / shopping,0,8,0,2017-09-18 11:33:11,0,0,11,33,30,9


In [29]:
# check for null values 
print ('no. of null values (dayofweek) : {}'.format(len(mtl_pts_id_trip_mode[mtl_pts_id_trip_mode['dayofweek'].isna()])))
print ('no. of null values (weekend) : {}'.format(len(mtl_pts_id_trip_mode[mtl_pts_id_trip_mode['weekend'].isna()])))
print ('no. of null values (start_hour) : {}'.format(len(mtl_pts_id_trip_mode[mtl_pts_id_trip_mode['start_hour'].isna()])))
print ('no. of null values (start_min) : {}'.format(len(mtl_pts_id_trip_mode[mtl_pts_id_trip_mode['start_min'].isna()])))
print ('no. of null values (start_hour_interval) : {}'.format(len(mtl_pts_id_trip_mode[mtl_pts_id_trip_mode['start_hour_interval'].isna()])))
print ('no. of null values (start_min_interval) : {}'.format(len(mtl_pts_id_trip_mode[mtl_pts_id_trip_mode['start_min_interval'].isna()])))

# convert to integer
mtl_pts_id_trip_mode['dayofweek'] = mtl_pts_id_trip_mode['dayofweek'].astype(int)
mtl_pts_id_trip_mode['weekend'] = mtl_pts_id_trip_mode['weekend'].astype(int)
mtl_pts_id_trip_mode['start_hour']= mtl_pts_id_trip_mode['start_hour'].astype(int)
mtl_pts_id_trip_mode['start_min'] = mtl_pts_id_trip_mode['start_min'].astype(int)
mtl_pts_id_trip_mode['start_hour_interval']= mtl_pts_id_trip_mode['start_hour_interval'].astype(int)
mtl_pts_id_trip_mode['start_min_interval'] = mtl_pts_id_trip_mode['start_min_interval'].astype(int)

no. of null values (dayofweek) : 0
no. of null values (weekend) : 0
no. of null values (start_hour) : 0
no. of null values (start_min) : 0
no. of null values (start_hour_interval) : 0
no. of null values (start_min_interval) : 0


In [30]:
# load nearest distance to bus stop info 
# bus stop info was calculated using qgis using nearest neighbour analysis
# source : https://www.qgistutorials.com/en/docs/3/nearest_neighbor_analysis.html

mtl_trajet_trip_start_csv = 'mtl_trajet_SMT_nearest_stop_dist_trip_start_MRP.csv'

mtl_trajet_trip_start = pd.read_csv(mtl_trajet_trip_start_csv)
mtl_trajet_trip_start = mtl_trajet_trip_start.rename(columns = {'HubName':'trip_start_stop_id', 'HubDist':'trip_start_stop_dist'})

# create discrete variable to indicate if a stop is within 100m, 250m, 500m away from start of trip
mtl_trajet_trip_start['trip_start_100m'] = np.where(mtl_trajet_trip_start['trip_start_stop_dist'] <= 100, 1, 0)
mtl_trajet_trip_start['trip_start_250m'] = np.where(mtl_trajet_trip_start['trip_start_stop_dist'] <= 250, 1, 0)
mtl_trajet_trip_start['trip_start_500m'] = np.where(mtl_trajet_trip_start['trip_start_stop_dist'] <= 500, 1, 0)

# merge with mtl_pts_id_trip_mode
mtl_pts_id_trip_mode = pd.merge(mtl_pts_id_trip_mode, mtl_trajet_trip_start, how = 'left', on = 'id_trip')

# check for null values
print ('no. of null values (trip_start_stop_id) : {}'.format(len(mtl_pts_id_trip_mode[mtl_pts_id_trip_mode['trip_start_stop_id'].isna()])))
print ('no. of null values (trip_start_stop_dist) : {}'.format(len(mtl_pts_id_trip_mode[mtl_pts_id_trip_mode['trip_start_stop_dist'].isna()])))
print ('no. of null values (trip_start_100m) : {}'.format(len(mtl_pts_id_trip_mode[mtl_pts_id_trip_mode['trip_start_100m'].isna()])))
print ('no. of null values (trip_start_250m) : {}'.format(len(mtl_pts_id_trip_mode[mtl_pts_id_trip_mode['trip_start_250m'].isna()])))
print ('no. of null values (trip_start_500m) : {}'.format(len(mtl_pts_id_trip_mode[mtl_pts_id_trip_mode['trip_start_500m'].isna()])))

mtl_pts_id_trip_mode.head()

no. of null values (trip_start_stop_id) : 0
no. of null values (trip_start_stop_dist) : 0
no. of null values (trip_start_100m) : 0
no. of null values (trip_start_250m) : 0
no. of null values (trip_start_500m) : 0


,id_trip,mode,purpose,mode_label,purpose_label,id_trip_seq,timestamp,dayofweek,weekend,start_hour,start_min,start_min_interval,start_hour_interval,trip_start_stop_id,trip_start_stop_dist,trip_start_100m,trip_start_250m,trip_start_500m
0,150744,automobile,drive / pick up a person,0,1,0,2017-09-18 10:03:52,0,0,10,3,0,9,50589,9.841453,1,1,1
1,173651,automobile,work / business meeting,0,9,0,2017-09-18 11:14:25,0,0,11,14,0,9,68,11994.541012,0,0,0
2,149086,public_transportation,work / business meeting,2,9,0,2017-09-18 11:22:11,0,0,11,22,15,9,52665,74.703198,1,1,1
3,358412,walk,work / business meeting,3,9,0,2017-09-18 11:29:54,0,0,11,29,15,9,51852,55.367950,1,1,1
4,35763,automobile,shopping / shopping,0,8,0,2017-09-18 11:33:11,0,0,11,33,30,9,56663,7632.963287,0,0,0


In [31]:
mtl_trajet_trip_end_csv = 'mtl_trajet_SMT_nearest_stop_dist_trip_end_MRP.csv'

mtl_trajet_trip_end = pd.read_csv(mtl_trajet_trip_end_csv)
mtl_trajet_trip_end = mtl_trajet_trip_end.rename(columns = {'HubName':'trip_end_stop_id', 'HubDist':'trip_end_stop_dist'})

mtl_trajet_trip_end['trip_end_100m'] = np.where(mtl_trajet_trip_end['trip_end_stop_dist'] <= 100, 1, 0)
mtl_trajet_trip_end['trip_end_250m'] = np.where(mtl_trajet_trip_end['trip_end_stop_dist'] <= 250, 1, 0)
mtl_trajet_trip_end['trip_end_500m'] = np.where(mtl_trajet_trip_end['trip_end_stop_dist'] <= 500, 1, 0)

# merge with mtl_pts_id_trip_mode
mtl_pts_id_trip_mode = pd.merge(mtl_pts_id_trip_mode, mtl_trajet_trip_end, how = 'left', on = 'id_trip')

# check for null values
print ('no. of null values (trip_end_stop_id) : {}'.format(len(mtl_pts_id_trip_mode[mtl_pts_id_trip_mode['trip_end_stop_id'].isna()])))
print ('no. of null values (trip_end_stop_dist) : {}'.format(len(mtl_pts_id_trip_mode[mtl_pts_id_trip_mode['trip_end_stop_dist'].isna()])))
print ('no. of null values (trip_end_100m) : {}'.format(len(mtl_pts_id_trip_mode[mtl_pts_id_trip_mode['trip_end_100m'].isna()])))
print ('no. of null values (trip_end_250m) : {}'.format(len(mtl_pts_id_trip_mode[mtl_pts_id_trip_mode['trip_end_250m'].isna()])))
print ('no. of null values (trip_end_500m) : {}'.format(len(mtl_pts_id_trip_mode[mtl_pts_id_trip_mode['trip_end_500m'].isna()])))

mtl_pts_id_trip_mode.head()

no. of null values (trip_end_stop_id) : 0
no. of null values (trip_end_stop_dist) : 0
no. of null values (trip_end_100m) : 0
no. of null values (trip_end_250m) : 0
no. of null values (trip_end_500m) : 0


,id_trip,mode,purpose,mode_label,purpose_label,id_trip_seq,timestamp,dayofweek,weekend,start_hour,start_min,start_min_interval,start_hour_interval,trip_start_stop_id,trip_start_stop_dist,trip_start_100m,trip_start_250m,trip_start_500m,trip_end_stop_id,trip_end_stop_dist,trip_end_100m,trip_end_250m,trip_end_500m
0,150744,automobile,drive / pick up a person,0,1,0,2017-09-18 10:03:52,0,0,10,3,0,9,50589,9.841453,1,1,1,53126,85.509602,1,1,1
1,173651,automobile,work / business meeting,0,9,0,2017-09-18 11:14:25,0,0,11,14,0,9,68,11994.541012,0,0,0,58324,5160.503865,0,0,0
2,149086,public_transportation,work / business meeting,2,9,0,2017-09-18 11:22:11,0,0,11,22,15,9,52665,74.703198,1,1,1,52666,15.631642,1,1,1
3,358412,walk,work / business meeting,3,9,0,2017-09-18 11:29:54,0,0,11,29,15,9,51852,55.367950,1,1,1,51752,12.205228,1,1,1
4,35763,automobile,shopping / shopping,0,8,0,2017-09-18 11:33:11,0,0,11,33,30,9,56663,7632.963287,0,0,0,56663,7619.812181,0,0,0


In [32]:
# load original gps coords for trip start and trip end calculate the straight line distance between origin and destination
mtl_trajet_trip_start_json = 'mtl_trajet_SMT_trip_start_MRP.geojson'
mtl_trajet_trip_end_json = 'mtl_trajet_SMT_trip_end_MRP.geojson'

mtl_trajet_trip_start_geo = gpd.read_file(mtl_trajet_trip_start_json)
mtl_trajet_trip_end_geo = gpd.read_file(mtl_trajet_trip_end_json)

mtl_trajet_trip_orig_dest = pd.merge(mtl_trajet_trip_start_geo, mtl_trajet_trip_end_geo, how = 'left', on = 'id_trip')

# calculate distance (km) between origin/destination
distance_series = calc_distance(mtl_trajet_trip_orig_dest)
# merge with mtl_trajet_trip_orig_dest
mtl_trajet_trip_orig_dest = merge_series_df(mtl_trajet_trip_orig_dest, [distance_series], ['distance_orig_dest'])
mtl_trajet_trip_orig_dest.head()

,id_trip,geometry_x,geometry_y,distance_orig_dest
0,150744,POINT (-73.64638 45.54629),POINT (-73.54532 45.55066),11.280322
1,173651,POINT (-73.81487 45.64468),POINT (-73.80019 45.55683),3.188922
2,149086,POINT (-73.56351 45.51645),POINT (-73.56520 45.50089),0.526382
3,358412,POINT (-73.59024 45.53261),POINT (-73.59427 45.52622),0.492388
4,35763,POINT (-73.47320 45.40843),POINT (-73.47267 45.40898),0.061052


In [33]:
mtl_pts_id_trip_mode = pd.merge(mtl_pts_id_trip_mode, mtl_trajet_trip_orig_dest, how = 'left', on = 'id_trip')

# del mtl_pts_id_trip_mode['geometry_x'] 	
# del mtl_pts_id_trip_mode['geometry_y']
print ('no. of null values (distance_orig_dest) : {}\n'.format(len(mtl_pts_id_trip_mode[mtl_pts_id_trip_mode['distance_orig_dest'].isna()])))

mtl_pts_id_trip_mode.head()

no. of null values (distance_orig_dest) : 0



,id_trip,mode,purpose,mode_label,purpose_label,id_trip_seq,timestamp,dayofweek,weekend,start_hour,start_min,start_min_interval,start_hour_interval,trip_start_stop_id,trip_start_stop_dist,trip_start_100m,trip_start_250m,trip_start_500m,trip_end_stop_id,trip_end_stop_dist,trip_end_100m,trip_end_250m,trip_end_500m,geometry_x,geometry_y,distance_orig_dest
0,150744,automobile,drive / pick up a person,0,1,0,2017-09-18 10:03:52,0,0,10,3,0,9,50589,9.841453,1,1,1,53126,85.509602,1,1,1,POINT (-73.64638 45.54629),POINT (-73.54532 45.55066),11.280322
1,173651,automobile,work / business meeting,0,9,0,2017-09-18 11:14:25,0,0,11,14,0,9,68,11994.541012,0,0,0,58324,5160.503865,0,0,0,POINT (-73.81487 45.64468),POINT (-73.80019 45.55683),3.188922
2,149086,public_transportation,work / business meeting,2,9,0,2017-09-18 11:22:11,0,0,11,22,15,9,52665,74.703198,1,1,1,52666,15.631642,1,1,1,POINT (-73.56351 45.51645),POINT (-73.56520 45.50089),0.526382
3,358412,walk,work / business meeting,3,9,0,2017-09-18 11:29:54,0,0,11,29,15,9,51852,55.367950,1,1,1,51752,12.205228,1,1,1,POINT (-73.59024 45.53261),POINT (-73.59427 45.52622),0.492388
4,35763,automobile,shopping / shopping,0,8,0,2017-09-18 11:33:11,0,0,11,33,30,9,56663,7632.963287,0,0,0,56663,7619.812181,0,0,0,POINT (-73.47320 45.40843),POINT (-73.47267 45.40898),0.061052


In [34]:
# get avg acceleration
mtl_pts_avg_acceleration = mtl_pts[['id_trip', 'acceleration']].groupby('id_trip')['acceleration'].mean().reset_index(name = 'avg_acceleration')
# get max acceleration
mtl_pts_max_acceleration = mtl_pts[['id_trip', 'acceleration']].groupby('id_trip')['acceleration'].max().reset_index(name = 'max_acceleration')
# merge acceleration
mtl_pts_acceleration = pd.merge(mtl_pts_avg_acceleration, mtl_pts_max_acceleration, how = 'left', on = 'id_trip')

# get avg speed
mtl_pts_avg_speed = mtl_pts[['id_trip', 'speed']].groupby('id_trip')['speed'].mean().reset_index(name = 'avg_speed')
# get max speed
mtl_pts_max_speed = mtl_pts[['id_trip', 'speed']].groupby('id_trip')['speed'].max().reset_index(name = 'max_speed')
# merge speed
mtl_pts_speed = pd.merge(mtl_pts_avg_speed, mtl_pts_max_speed, how = 'left', on = 'id_trip')

# merge acceleration + speed
mtl_pts_accel_speed = pd.merge(mtl_pts_acceleration, mtl_pts_speed, how = 'left', on = 'id_trip')


# merge with mtl_pts_id_trip_mode
mtl_pts_id_trip_mode = pd.merge(mtl_pts_id_trip_mode, mtl_pts_accel_speed, how = 'left', on = 'id_trip')

print ('no. of null values (avg_acceleration) : {}'.format(len(mtl_pts_id_trip_mode[mtl_pts_id_trip_mode['avg_acceleration'].isna()])))
print ('no. of null values (max_acceleration) : {}'.format(len(mtl_pts_id_trip_mode[mtl_pts_id_trip_mode['max_acceleration'].isna()])))
print ('no. of null values (avg_speed) : {}'.format(len(mtl_pts_id_trip_mode[mtl_pts_id_trip_mode['avg_speed'].isna()])))
print ('no. of null values (max_speed) : {}\n'.format(len(mtl_pts_id_trip_mode[mtl_pts_id_trip_mode['max_speed'].isna()])))

mtl_pts_id_trip_mode.head()

no. of null values (avg_acceleration) : 0
no. of null values (max_acceleration) : 0
no. of null values (avg_speed) : 0
no. of null values (max_speed) : 0



,id_trip,mode,purpose,mode_label,purpose_label,id_trip_seq,timestamp,dayofweek,weekend,start_hour,start_min,start_min_interval,start_hour_interval,trip_start_stop_id,trip_start_stop_dist,trip_start_100m,trip_start_250m,trip_start_500m,trip_end_stop_id,trip_end_stop_dist,trip_end_100m,trip_end_250m,trip_end_500m,geometry_x,geometry_y,distance_orig_dest,avg_acceleration,max_acceleration,avg_speed,max_speed
0,150744,automobile,drive / pick up a person,0,1,0,2017-09-18 10:03:52,0,0,10,3,0,9,50589,9.841453,1,1,1,53126,85.509602,1,1,1,POINT (-73.64638 45.54629),POINT (-73.54532 45.55066),11.280322,-0.000079,0.001860,15.002625,25.900000
1,173651,automobile,work / business meeting,0,9,0,2017-09-18 11:14:25,0,0,11,14,0,9,68,11994.541012,0,0,0,58324,5160.503865,0,0,0,POINT (-73.81487 45.64468),POINT (-73.80019 45.55683),3.188922,-0.000066,0.001358,16.807375,30.760000
2,149086,public_transportation,work / business meeting,2,9,0,2017-09-18 11:22:11,0,0,11,22,15,9,52665,74.703198,1,1,1,52666,15.631642,1,1,1,POINT (-73.56351 45.51645),POINT (-73.56520 45.50089),0.526382,-0.000017,0.000239,1.358143,3.550000
3,358412,walk,work / business meeting,3,9,0,2017-09-18 11:29:54,0,0,11,29,15,9,51852,55.367950,1,1,1,51752,12.205228,1,1,1,POINT (-73.59024 45.53261),POINT (-73.59427 45.52622),0.492388,-0.000001,0.000038,1.611429,2.410000
4,35763,automobile,shopping / shopping,0,8,0,2017-09-18 11:33:11,0,0,11,33,30,9,56663,7632.963287,0,0,0,56663,7619.812181,0,0,0,POINT (-73.47320 45.40843),POINT (-73.47267 45.40898),0.061052,-0.000111,0.000779,7.753078,14.781643


In [35]:
len(mtl_pts_id_trip_mode)

46311

In [36]:
# save to csv
mtl_pts_id_trip_mode.to_csv('mtl_pts_2017_aux_dimensions_MRP_NEW.csv')

In [ ]:
# weather station data
# change directory
drive.mount('/content/drive', force_remount= True)
os.chdir('/content/drive/My Drive/MRP/environment_canada_historical_weather')

Mounted at /content/drive


In [ ]:
mtl_weather_hourly = 'mtl_weather_2017_hourly.csv'

# import hourly data
mtl_hourly = pd.read_csv(mtl_weather_hourly, index_col = 0)
mtl_hourly.head()

,station_name,station_id,station_operator,prov,lat,lon,elev,climate_id,WMO_id,TC_id,date,time,year,month,day,hour,weather,hmdx,hmdx_flag,pressure,pressure_flag,rel_hum,rel_hum_flag,temp,temp_dew,temp_dew_flag,temp_flag,visib,visib_flag,wind_chill,wind_chill_flag,wind_dir,wind_dir_flag,wind_spd,wind_spd_flag
1,MCTAVISH,10761,NaN,QC,45.5,-73.58,72.8,7024745,71612.0,WTA,2017-09-18,2017-09-18 00:00:00,2017,9,18,00:00,NaN,25.0,NaN,101.01,NaN,72.0,NaN,20.9,15.7,NaN,NaN,NaN,NaN,NaN,NaN,21.0,NaN,2.0,NaN
2,MCTAVISH,10761,NaN,QC,45.5,-73.58,72.8,7024745,71612.0,WTA,2017-09-18,2017-09-18 01:00:00,2017,9,18,01:00,NaN,26.0,NaN,101.01,NaN,77.0,NaN,20.8,16.5,NaN,NaN,NaN,NaN,NaN,NaN,22.0,NaN,3.0,NaN
3,MCTAVISH,10761,NaN,QC,45.5,-73.58,72.8,7024745,71612.0,WTA,2017-09-18,2017-09-18 02:00:00,2017,9,18,02:00,NaN,NaN,NaN,101.01,NaN,85.0,NaN,19.8,17.1,NaN,NaN,NaN,NaN,NaN,NaN,31.0,NaN,2.0,NaN
4,MCTAVISH,10761,NaN,QC,45.5,-73.58,72.8,7024745,71612.0,WTA,2017-09-18,2017-09-18 03:00:00,2017,9,18,03:00,NaN,NaN,NaN,101.01,NaN,88.0,NaN,18.8,16.8,NaN,NaN,NaN,NaN,NaN,NaN,23.0,NaN,2.0,NaN
5,MCTAVISH,10761,NaN,QC,45.5,-73.58,72.8,7024745,71612.0,WTA,2017-09-18,2017-09-18 04:00:00,2017,9,18,04:00,NaN,NaN,NaN,101.01,NaN,87.0,NaN,18.6,16.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [ ]:
# convert hourly format to integer
mtl_hourly['hour'] = pd.to_datetime(mtl_hourly['time']).dt.hour
mtl_hourly['hour'].head()

1    0
2    1
3    2
4    3
5    4
Name: hour, dtype: int64

In [ ]:
# create keys (year, month, day, hour) in order to join weather station data with mtl_pts
mtl_pts['timestamp'] = pd.to_datetime(mtl_pts['timestamp'])

# get year/month/day/hour
mtl_pts['year'] = mtl_pts['timestamp'].dt.year
mtl_pts['month'] = mtl_pts['timestamp'].dt.month
mtl_pts['day'] = mtl_pts['timestamp'].dt.day
mtl_pts['hour'] = mtl_pts['timestamp'].dt.hour

In [ ]:
# merge weather station data with mtl_pts
mtl_pts = mtl_pts.merge(mtl_hourly[['station_id', 'year', 'month', 'day', 'hour', 'pressure', 'temp', 'temp_dew', 'wind_spd']], 
                                    how = 'left', 
                                    left_on = ['weather_station_id', 'year', 'month', 'day', 'hour'], 
                                    right_on = ['station_id', 'year', 'month', 'day', 'hour'])

In [ ]:
# ensure that there are no missing variables
print ('no. of missing records (pressure) : {}'.format(len(mtl_pts[mtl_pts['pressure'].isna()])))
print ('no. of missing records (temp) : {}'.format(len(mtl_pts[mtl_pts['temp'].isna()])))
print ('no. of missing records (temp_dew) : {}'.format(len(mtl_pts[mtl_pts['temp_dew'].isna()])))
print ('no. of missing records (wind_spd) : {}'.format(len(mtl_pts[mtl_pts['wind_spd'].isna()])))

no. of missing records (pressure) : 0
no. of missing records (temp) : 0
no. of missing records (temp_dew) : 0
no. of missing records (wind_spd) : 0


In [ ]:
# get max sequence value (padding)
max_seq = mtl_pts['id_trip_seq'].max() + 1
print ('max sequence : {}'.format(max_seq))

max sequence : 80


In [ ]:
# query id_trip
id_trip = mtl_pts['id_trip'].unique()
print ('no. of trips : {}'.format(len(id_trip)))

attributes = ['speed', 'acceleration', 'jerk', 'bearing', 'DAUID', 'CTUID', 'pressure', 'temp', 'temp_dew', 'wind_spd']

speed = []
acceleration = []
jerk = []
bearing = []
DAUID = []
CTUID = []
pressure = []
temp = []
temp_dew = []
wind_spd = []

for id in id_trip:
  data = mtl_pts[attributes][mtl_pts['id_trip'] == id]
  speed.append(data[attributes[0]].tolist())
  acceleration.append(data[attributes[1]].tolist())
  jerk.append(data[attributes[2]].tolist())
  bearing.append(data[attributes[3]].tolist())
  DAUID.append(data[attributes[4]].tolist())
  CTUID.append(data[attributes[5]].tolist())
  pressure.append(data[attributes[6]].tolist())
  temp.append(data[attributes[7]].tolist())
  temp_dew.append(data[attributes[8]].tolist())
  wind_spd.append(data[attributes[9]].tolist())



no. of trips : 42450


In [ ]:
drive.mount('/content/drive', force_remount= True)
os.chdir('/content/drive/My Drive/MRP/to_be_submitted/dimensions_non_padded_min20')

Mounted at /content/drive


In [ ]:
# save non-padded arrays
with open('mtl_pts_2017_MODEL_speed_non_padded_min20_MRP.csv', 'w') as csvFile:
    for row in speed:
        csvFile.write('%s\n' % row)

with open('mtl_pts_2017_MODEL_acceleration_non_padded_min20_MRP.csv', 'w') as csvFile:
    for row in acceleration:
        csvFile.write('%s\n' % row)

with open('mtl_pts_2017_MODEL_jerk_non_padded_min20_MRP.csv', 'w') as csvFile:
    for row in jerk:
        csvFile.write('%s\n' % row)

with open('mtl_pts_2017_MODEL_bearing_non_padded_min20_MRP.csv', 'w') as csvFile:
    for row in bearing:
        csvFile.write('%s\n' % row)

with open('mtl_pts_2017_MODEL_DAUID_non_padded_min20_MRP.csv', 'w') as csvFile:
    for row in DAUID:
        csvFile.write('%s\n' % row)

with open('mtl_pts_2017_MODEL_CTUID_non_padded_min20_MRP.csv', 'w') as csvFile:
    for row in CTUID:
        csvFile.write('%s\n' % row)

with open('mtl_pts_2017_MODEL_pressure_non_padded_min20_MRP.csv', 'w') as csvFile:
    for row in CTUID:
        csvFile.write('%s\n' % row)

with open('mtl_pts_2017_MODEL_temp_non_padded_min20_MRP.csv', 'w') as csvFile:
    for row in temp:
        csvFile.write('%s\n' % row)

with open('mtl_pts_2017_MODEL_temp_dew_non_padded_min20_MRP.csv', 'w') as csvFile:
    for row in temp_dew:
        csvFile.write('%s\n' % row)

with open('mtl_pts_2017_MODEL_wind_spd_non_padded_min20_MRP.csv', 'w') as csvFile:
    for row in wind_spd:
        csvFile.write('%s\n' % row)


In [ ]:
# pad sequences for each dimension
speed_pad = pad_sequences(speed, maxlen= max_seq, dtype = 'float64', padding = 'post', value = 0.0)
acceleration_pad = pad_sequences(acceleration, maxlen= max_seq, dtype = 'float64', padding = 'post', value = 0.0)
jerk_pad = pad_sequences(jerk, maxlen= max_seq, dtype = 'float64', padding = 'post', value = 0.0)
bearing_pad = pad_sequences(bearing, maxlen= max_seq, dtype = 'float64', padding = 'post', value = 0.0)

DAUID_pad = pad_sequences(DAUID, maxlen= max_seq, dtype = 'float64', padding = 'post', value = 0.0)
CTUID_pad = pad_sequences(CTUID, maxlen= max_seq, dtype = 'float64', padding = 'post', value = 0.0)

pressure_pad = pad_sequences(pressure, maxlen= max_seq, dtype = 'float64', padding = 'post', value = 0.0)
temp_pad = pad_sequences(temp, maxlen= max_seq, dtype = 'float64', padding = 'post', value = 0.0)
temp_dew_pad = pad_sequences(temp_dew, maxlen= max_seq, dtype = 'float64', padding = 'post', value = 0.0)
wind_spd_pad = pad_sequences(wind_spd, maxlen= max_seq, dtype = 'float64', padding = 'post', value = 0.0)

print ('speed_pad shape : {} '.format(speed_pad.shape))
print ('acceleration_pad shape : {}'.format(acceleration_pad.shape))
print ('jerk_pad shape : {}'.format(jerk_pad.shape))
print ('bearing pad shape : {}'.format(bearing_pad.shape))
print ('DAUID pad shape : {}'.format(DAUID_pad.shape))
print ('CTUID pad shape : {}'.format(CTUID_pad.shape))
print ('pressure pad shape : {}'.format(pressure_pad.shape))
print ('temp pad shape : {}'.format(temp_pad.shape))
print ('temp dew pad shape : {}'.format(temp_dew_pad.shape))
print ('wind spd pad shape : {}'.format(wind_spd_pad.shape))

speed_pad shape : (42450, 80) 
acceleration_pad shape : (42450, 80)
jerk_pad shape : (42450, 80)
bearing pad shape : (42450, 80)
DAUID pad shape : (42450, 80)
CTUID pad shape : (42450, 80)
pressure pad shape : (42450, 80)
temp pad shape : (42450, 80)
temp dew pad shape : (42450, 80)
wind spd pad shape : (42450, 80)


In [ ]:
# change directory
drive.mount('/content/drive', force_remount= True)
os.chdir('/content/drive/My Drive/MRP/to_be_submitted/dimensions_80_min20')

# save each 2D array to csv
np.savetxt('mtl_pts_2017_MODEL_speed_min20_MRP.csv', speed_pad, delimiter = ',')
print ('speed SAVED')

np.savetxt('mtl_pts_2017_MODEL_acceleration_min20_MRP.csv', acceleration_pad, delimiter = ',')
print ('acceleration SAVED')

np.savetxt('mtl_pts_2017_MODEL_jerk_min20_MRP.csv', jerk_pad, delimiter = ',')
print ('jerk SAVED')

np.savetxt('mtl_pts_2017_MODEL_bearing_min20_MRP.csv', bearing_pad, delimiter = ',')
print ('bearing SAVED')

np.savetxt('mtl_pts_2017_MODEL_DAUID_min20_MRP.csv', DAUID_pad, delimiter = ',')
print ('DAUID SAVED')

np.savetxt('mtl_pts_2017_MODEL_CTUID_min20_MRP.csv', CTUID_pad, delimiter = ',')
print ('CTUID SAVED')

np.savetxt('mtl_pts_2017_MODEL_pressure_min20_MRP.csv', pressure_pad, delimiter = ',')
print ('pressure SAVED')

np.savetxt('mtl_pts_2017_MODEL_temp_min20_MRP.csv', temp_pad, delimiter = ',')
print ('temp SAVED')

np.savetxt('mtl_pts_2017_MODEL_temp_dew_min20_MRP.csv', temp_dew_pad, delimiter = ',')
print ('temp dew SAVED')

np.savetxt('mtl_pts_2017_MODEL_wind_spd_min20_MRP.csv', wind_spd_pad, delimiter = ',')
print ('wind spd SAVED')


Mounted at /content/drive
speed SAVED
acceleration SAVED
jerk SAVED
bearing SAVED
DAUID SAVED
CTUID SAVED
pressure SAVED
temp SAVED
temp dew SAVED
wind spd SAVED


In [ ]:
# check
mtl_pts['speed'][mtl_pts['id_trip'] == 358412]

230    1.99
231    1.71
232    1.59
233    1.58
234    1.58
235    1.62
236    1.59
237    1.67
238    1.89
239    2.41
240    1.63
241    1.63
242    1.63
243    1.56
244    1.49
245    1.59
246    1.47
247    1.56
248    1.52
249    1.54
250    1.58
251    1.58
252    1.58
253    1.29
254    1.36
255    1.68
256    1.42
257    1.38
Name: speed, dtype: float64

In [ ]:
speed_pad[3]

array([1.99, 1.71, 1.59, 1.58, 1.58, 1.62, 1.59, 1.67, 1.89, 2.41, 1.63,
       1.63, 1.63, 1.56, 1.49, 1.59, 1.47, 1.56, 1.52, 1.54, 1.58, 1.58,
       1.58, 1.29, 1.36, 1.68, 1.42, 1.38, 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  ])

####**transport mode**

In [ ]:
# change directory
drive.mount('/content/drive', force_remount= True)
os.chdir('/content/drive/My Drive/MRP/mtl trajet data')

Mounted at /content/drive


In [ ]:
mtl_trajet_json = 'trajets_mtl_trajet_2017-1.geojson'

# import file
mtl_trajet = gpd.read_file(mtl_trajet_json)
mtl_trajet.head()

,id_trip,starttime,endtime,mode,purpose,geometry
0,1547,2017-09-18T04:16:58,2017-09-18T04:26:26,None,None,"MULTILINESTRING ((-73.65117 45.54450, -73.6512..."
1,308312,2017-09-18T06:17:46,2017-09-18T06:58:49,None,None,"MULTILINESTRING ((-73.65099 45.54450, -73.6510..."
2,384772,2017-09-18T09:30:24,2017-09-18T10:17:28,None,None,"MULTILINESTRING ((-73.83287 45.63607, -73.8317..."
3,150744,2017-09-18T10:02:50,2017-09-18T10:17:12,Voiture / Moto,Reconduire / aller chercher une personne,"MULTILINESTRING ((-73.64638 45.54629, -73.6460..."
4,199011,2017-09-18T10:18:40,2017-09-18T10:27:44,None,None,"MULTILINESTRING ((-73.74550 45.55839, -73.7454..."


In [ ]:
transport_mode_fr = ['Voiture / Moto', 'Transport collectif', 'À pied', 'Vélo', 'Autopartage', 'Autre', 'Taxi', 'ND']
transport_mode_en = ['automobile', 'public_transportation', 'walk', 'bike', 'ride_share', 'other', 'taxi', 'na']

mtl_trajet['mode'].replace(to_replace = transport_mode_fr, value = transport_mode_en, inplace = True)

In [ ]:
# merge transport modes together
mtl_trajet['mode'] = mtl_trajet['mode'].str.replace('ride_share', 'automobile')
mtl_trajet['mode'] = mtl_trajet['mode'].str.replace('taxi', 'automobile')

mtl_trajet['mode'].unique()

array([None, 'automobile', 'À pied, Transport collectif',
       'public_transportation', 'walk', 'bike',
       'À pied, Transport collectif, Vélo',
       'Transport collectif, Voiture / Moto', 'À pied, Vélo',
       'À pied, Voiture / Moto, Vélo', 'other',
       'Transport collectif, Vélo',
       'À pied, Transport collectif, Voiture / Moto',
       'Transport collectif, Voiture / Moto, Vélo',
       'À pied, Voiture / Moto', 'Voiture / Moto, Vélo', 'na',
       'Taxi, Vélo', 'À pied, Autopartage, Vélo',
       'Autopartage, Transport collectif, Voiture / Moto',
       'Taxi, Voiture / Moto', 'À pied, Autopartage, Voiture / Moto',
       'Autopartage, Autre', 'À pied, Taxi, Vélo', 'Autopartage, Taxi',
       'Autopartage, Voiture / Moto',
       'Autopartage, Autre, Voiture / Moto',
       'Autopartage, Transport collectif',
       'À pied, Transport collectif, Voiture / Moto, Vélo',
       'Autre, Transport collectif', 'À pied, Autre, Transport collectif',
       'Autre, Vélo', '

In [ ]:
mtl_trajet_mode = mtl_trajet[['id_trip', 'mode', 'purpose']]

In [ ]:
mtl_trajet_mode['mode'].unique()

array([None, 'automobile', 'À pied, Transport collectif',
       'public_transportation', 'walk', 'bike',
       'À pied, Transport collectif, Vélo',
       'Transport collectif, Voiture / Moto', 'À pied, Vélo',
       'À pied, Voiture / Moto, Vélo', 'other',
       'Transport collectif, Vélo',
       'À pied, Transport collectif, Voiture / Moto',
       'Transport collectif, Voiture / Moto, Vélo',
       'À pied, Voiture / Moto', 'Voiture / Moto, Vélo', 'na',
       'Taxi, Vélo', 'À pied, Autopartage, Vélo',
       'Autopartage, Transport collectif, Voiture / Moto',
       'Taxi, Voiture / Moto', 'À pied, Autopartage, Voiture / Moto',
       'Autopartage, Autre', 'À pied, Taxi, Vélo', 'Autopartage, Taxi',
       'Autopartage, Voiture / Moto',
       'Autopartage, Autre, Voiture / Moto',
       'Autopartage, Transport collectif',
       'À pied, Transport collectif, Voiture / Moto, Vélo',
       'Autre, Transport collectif', 'À pied, Autre, Transport collectif',
       'Autre, Vélo', '

In [ ]:
mtl_pts = mtl_pts.merge(mtl_trajet_mode, how = 'left', on = 'id_trip')

In [ ]:
mtl_pts.head()

,id_trip,id_trip_seq,timestamp,DAUID,CTUID,weather_station_id,distance,speed,acceleration,jerk,bearing,geometry,year,month,day,hour,station_id,pressure,temp,temp_dew,wind_spd,mode,purpose
0,150744,0,2017-09-18 10:03:52,24660491,4620266.00,10761,0.062391,8.31,0.000354,-0.000130,0.001470,MULTIPOINT (-73.64608 45.54617),2017,9,18,10,10761,101.16,19.8,16.7,7.0,automobile,Reconduire / aller chercher une personne
1,150744,1,2017-09-18 10:03:57,24660491,4620266.00,10761,0.085496,13.60,-0.000294,-0.000050,0.000854,MULTIPOINT (-73.64534 45.54595),2017,9,18,10,10761,101.16,19.8,16.7,7.0,automobile,Reconduire / aller chercher une personne
2,150744,2,2017-09-18 10:04:03,24660491,4620266.00,10761,0.062440,14.79,-0.000594,0.000227,0.000819,MULTIPOINT (-73.64433 45.54566),2017,9,18,10,10761,101.16,19.8,16.7,7.0,automobile,Reconduire / aller chercher une personne
3,150744,3,2017-09-18 10:04:08,24660491,4620266.00,10761,0.066643,11.87,0.000542,-0.000072,0.000596,MULTIPOINT (-73.64359 45.54545),2017,9,18,10,10761,101.16,19.8,16.7,7.0,automobile,Reconduire / aller chercher une personne
4,150744,4,2017-09-18 10:04:15,24660491,4620266.00,10761,0.053257,11.63,0.000040,-0.000164,0.000135,MULTIPOINT (-73.64280 45.54523),2017,9,18,10,10761,101.16,19.8,16.7,7.0,automobile,Reconduire / aller chercher une personne


In [ ]:
mtl_pts['mode'].unique()

array(['automobile', 'public_transportation', 'walk', 'bike'],
      dtype=object)

In [ ]:
modes = ['automobile', 'public_transportation', 'walk', 'bike', 'other', 'na']


In [ ]:
export = mtl_trajet_mode[mtl_trajet_mode['mode'].isin(modes)]

In [ ]:
# change directory
drive.mount('/content/drive', force_remount= True)
os.chdir('/content/drive/My Drive/MRP/to_be_submitted')

Mounted at /content/drive


In [ ]:
export.to_csv('mtl_trajet_2017_transport_mode_min20_MRP.csv')